## Last Presentation  summray : 

### VoxelNet Architecture 

:::: {.columns}

::: {.column width="70%"}
![](images/Acrobat_ZOIOLjvhTa.png)
:::

::: {.column width="30%"}
The VoxelNet consists of three functional blocks

1. Feature learning network
2. Convolutional middle layers
3. Region proposal network
:::

::::




## Last Presentation  summray :  
### Feature Learning Network 

:::: {.columns}

::: {.column width="60%"}
### Voxel Partition

- The  3D voxel grid is of size $D'=D/v_D, H'=H/v_H,  W'=W/v_W$ where $v_D,v_H,v_W$ defines the voxel size accordingly , where $D, H, W$  are point clouds across $Z, Y, X$ axes respectively.
- Point density in LiDAR point clouds is variable, resulting in varying numbers of points within each voxel.
:::

::: {.column width="40%"}
![](images/Acrobat_ffA1dvD6zy.png)
:::

::::

## Last Presentation  summray : 
### Convolutional Middle Layers 

- The convolutional middle layers aggregate voxel-wise features within a __progressively expanding receptive field, adding more context to the shape description__
- Depending on the task this layer' details can vary.

<br><br>
![](images/chrome_jVqU0WaxpR.png)

## Last Presentation  summray : 
### Region Proposal Network 
 
 - RPN is combined with  the feature learning network and convolutional middle layers to form an __end-to-end trainable pipeline__.<br>
   ![](images/Acrobat_gayrjAQFrA.png)<br>
- The first layer of each block downsamples the feature map by half.
- The output of each block is upsampled to a fixed size and concatenated to construct a high-resolution feature map.

## Last Presentation  summray :  
### Loss Function 

- Consider ground truth  $(x_{c}^{g},y_{c}^{g},z_{c}^{g},l^{g},w^{g},h^{g},\theta^{g}),$ where $x_{c}^{g},y_{c}^{g},z_{c}^{g}$ represent the center location, $l^{g},w^{g},h^{g}$ are length, width, height of the box,${\theta}^{g}$ is the yaw rotation.Consider Anchor   $(x_{c}^{a},y_{c}^{a},z_{c}^{a},l^{a},w^{a},h^{a},\theta^{a})$ A vector $\mathbf{u}^{*}\in\mathbb{R}^{7}$ containing below 7 elements are calculated as below:
  $$\begin{align*}
      &\Delta x={\frac{x_{c}^{g}-x_{c}^{a}}{d^{a}}},\Delta y={\frac{y_{c}^{g}-y_{c}^{a}}{d^{a}}},\Delta z={\frac{z_{c}^{g}-z_{c}^{a}}{h^{a}}},\\
      &\Delta l=\log(\frac{l^{g}}{l^{a}}),\Delta w=\log(\frac{w^{g}}{w^{a}}),\Delta h=\log(\frac{h^{g}}{h^{a}}), \tag{1} \\
      &\Delta\theta=\theta^{g}-\theta^{a}
  \end{align*}$$
  Where  $\;d^{a}\,=\,\sqrt{(l^{a})^{2}+(w^{a})^{2}}$



##   

Loss Function cont ...

- Now we find loss as shown below 
  $$\begin{align*}
     L&=\alpha{\frac{1}{N_{\mathrm{pos}}}}\sum_{i}L_{\mathrm{cls}}(p_{i}^{\mathrm{pos}},1)\\ 
     &+\beta{\frac{1}{N_{\mathrm{neg}}}}\sum_{j}L_{\mathrm{cls}}(p_{j}^{\mathrm{neg}},0)\\ 
     &+\frac{1}{N_{\mathrm{pos}}}\sum_{i}L_{\mathrm{reg}}({\bf u}_{i},{\bf u}_{i}^{*}) \tag{2}
  \end{align*}$$
- where $p_{i}^{\mathrm{pos}}$ and $p_{j}^{\mathrm{neg}}$ represent the softmax output for positive anchor $a_{i}^{\mathrm{pos}}$ and negative anchor $a_{j}^{\mathrm{neg}}$ respectively,
- $\mathbf{u}_{i}\ \in\ \mathbb{R}^{7}$ and $\mathbf{u}_{i}^{*}\ \in\ \mathbb{R}^{7}$ are the regression output and ground truth for positive anchor $a_{i}^{\mathrm{pos}}$

## Implementation : 
### Network Details  


- $v_{D}\,=\,0.4,v_{H}\,=\,0.2,v_{W}\,=\,0.2$
- $D^{\prime}\;=\;10,\;H^{\prime}\;=\;400,\;W^{\prime}\;=\;352$
- The maximum number of randomly sampled points in each non-empty voxel $T\,=\,35$
- For $\mathrm{VFE}-1(7, 32)$ and and $\mathrm{VFE}-2(32, 128)$ was used.
- The final $\mathrm{FCN}$  maps $\mathrm{VFE}$ output to $\mathbb{R}^{128}$
- Thus the feature learning net generates a sparse tensor of shape $128\times10\times400\times352$


##  
 
 Network Details, Car Detection cont ...

 ![](images/Acrobat_9Ud4aJ0aiX.png)

## Implementation : 

### Data set details

- KITTI 3D object detection dataset is used,which contains 7,481 training images/point clouds and 7,518 test images/point clouds
- The data has three categories : Car, Pedestrian, and Cyclist, for each class data is divided in the three difficulty levels : easy, moderate, and hard

:::: {.columns}

::: {.column width="40%"}
Training Dataset<br>
length : 3712



| category   | number |
|------------|--------|
| Pedestrian | 2207   |
| Cyclist    | 734    |
| Car        | 14357  |


:::

::: {.column width="60%"}
Validation  Dataset<br>
length: 3769


| category   | number |
|------------|--------|
| Pedestrian | 2280   |
| Cyclist    | 893    |
| Car        | 14385  |


:::

::::

## Implementation : 

### Training  

 - Above model was implemented using the below environment:
   - PyTorch: 1.7.0
   - CUDA Runtime 10.2
   - Python: 3.7.16
   - CuDNN 7.6.5
   - TorchVision: 0.8.1
   - MMEngine: 0.7.2
   - MMDetection3D: 1.1.0rc3
 - A similar network with different parameters, was used for Pedestrian and Cyclist Detection
    - 2 anchors, smaller anchors, small stride to capture fine details etc.


## Implementation : 

### Training  

- The training was done with the exact same hyper parameter as suggested by the authors: 
    - During training, stochastic gradient descent (SGD) with learning rate 0.01 was used for the first 150 epochs and decreased the learning rate to 0.001 for the last 10 epochs.
- Data Augmentation 
    - perturbation independently to each ground truth 3D bounding box together with those LiDAR points within the box.
    - Global scaling.

## Implementation : Evaluation

 
- Author follows official KITTI evaluation protocol, which is the most appropriate method to capture object detection performance.
- As we can't find True negative in case of object detection so we do not report confusion matrix.
- For the same reason we can't find ROC curve, Hence for object detection evaluation we go for Precision-Recall curve, the curve doesn't involve True negative so it is possible to compute.
- We first generate prediction using model and find class label then find IOU, precision, recall
- Find the area under precision recall curve. which is known as the  Average precision.
- There are many method to find the area under this PR curve, for eg. AP11, AP40. 


## Implementation : Results 
:::: {.columns}

::: {.column width="50%"}

From Implementation

|  Car   | Easy  | Moderate | Hard  | |
|-----|-------|----------|-------|-|
| BEV | 87.50 | 83.80    | 77.37 | |
| 3D  | 80.96 | 64.45    | 61.75 | |


|  Ped   | Easy  | Moderate | Hard  | |
|-----|-------|----------|-------|-|
| BEV | 66.94 | 60.16    | 57.79 | |
| 3D  | 56.36 | 54.44    | 49.56 | |


|  Cyc   | Easy  | Moderate | Hard  | |
|-----|-------|----------|-------|-|
| BEV | 73.34 | 51.23    | 51.24 | |
| 3D  | 66.23 | 46.32    | 46.23 | |

:::

::: {.column width="50%"}

From Paper

| |  Car   | Easy  | Moderate | Hard  |
|-|-----|-------|----------|-------|
| | BEV | 89.60 | 84.81    | 78.57 |
| | 3D  | 81.97 | 65.46    | 62.85 |


| |  Ped   | Easy  | Moderate | Hard  |
|-|-----|-------|----------|-------|
| | BEV | 65.95 | 61.05    | 56.98 |
| | 3D  | 57.86 | 53.42    | 48.87 |


| |   Cyc  | Easy  | Moderate | Hard  |
|-|-----|-------|----------|-------|
| | BEV | 74.41 | 52.18    | 50.49 |
| | 3D  | 67.17 | 47.65    | 45.11 |

:::

::::



## Novelty  

Following Novelty were proposed:

- We do not need to predict separately  for positive and negative.
- VFE can be improved with the help of another encoder ( eg. transformer ) 
- One universal network instead of two.
- Extend  VoxelNet for joint LiDAR, Radar and image-based end-to-end 3D detection.
- Previous time frames can be taken into account improve the detection. 

Among all these, the below 3 was implemented, others can be implemented 

- We do not need to predict separately  for positive and negative.
- One universal network instead of two.
- VFE can be improved with the help of another encoder ( point pillar ) 


## 1. Novelty  


- The Original voxel net has two separate head for predicting Negative and Positive class classification score, 
- It was changed to  have just one output for classification head  and cross entropy loss was used.
- Finally I changed the cross entropy loss to focal loss^[Tsung-Yi Lin, Priya Goyal, Ross Girshick, Kaiming He, Piotr Dollár], While doing hyper parameter tuning, I observed that the cyclist class was not performing well, as we can notice this class has only 734 sample in the training data, which is much lesser compared to other class, 
- It was found that for gamma = 2.0 and alpha = 0.25 the model performs the best.
  $$FL(p_t)=-\alpha(1-p_t)^\gamma\log (p_t)$$
  Here $p_t$ is the target class, $\gamma=2.0$ and $\alpha=0.25$

## 2. Novelty 

- It was found that instead of training two network one for car, and another for pedestrian and cyclist, it is possible to train just one network if we carefully adjust stride in the network.
- Following Point pillars network^[alex, sourabh, holger, lubing, jiong.yang, oscar], I adjusted the stride of the network to 2 for back bone layer, and anchor size as well, and found that the the model was performing good on just one common network architecture. 
- It can also be noticed that yolo has 1000s of class and just one network for the classes. Yolo does it by careful selection of anchor box.
 

## 3. Novelty 

:::: {.columns}

::: {.column width="40%"}

- For the voxel encoder part we again follow point pillars network
- We borrow only encoder form point pillars not the whole network.
- point pillars uses SSD as feature extractor we use RPN as suggested by voxelnet.
- Point pillar encodes voxel very differently and more efficiently as compared to voxelnet

:::

::: {.column width="60%"}

![](images/chrome_W2rYqD4tqm.png) 

::: {style="font-size: 50%;"}

Courtesy: [medium](https://becominghuman.ai/pointpillars-3d-point-clouds-bounding-box-detection-and-tracking-pointnet-pointnet-lasernet-67e26116de5a)
:::

:::

::::



  

## Implementation

:::: {.columns}

::: {.column width="70%"}
![](images/Acrobat_8nIjIYkFMb.png)
:::

::: {.column width="30%"}
- consider $l$ a point in the point cloud with coordinates of $x,y,z$ and reflectance $r$.
:::

::::


- Now discretize the point cloud into evenly spaced $x,y$ plane creating a set of pillars $\mathcal P$, $|\mathcal P|=B$
- There is no hyper parameter in $z$ dimension
- Now find $x_c, y_c, z_c, x_p,y_p$ where the $c$ subscript denotes
distance to the arithmetic mean of all points in the pillar and
the $p$ subscript denotes the offset from the pillar $x, y$ center.
- Now the lidar point is 9 dimensional $D=9$ 

## Implementation

Point pillar network mentioned $D=9$  but in reality while implementing we need to include reflectance as well so it is actually $10$.
- Next, we use a simplified version of PointNet to generate a $(C, P,N)$ sized tensor.
- This is followed by a max operation over the channels to create an output tensor of size $(C, P)$
  ![](images/chrome_JcI6AbcymW.png)


## Implementation

- The point piller is just a simple version of point net 

  ```python
  PillarFeatureNet(
    (pfn_layers): ModuleList(
      (0): PFNLayer(
        (norm): BatchNorm1d(64, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
        (linear): Linear(in_features=10, out_features=64, bias=False)
      )
    )
  )
  ```
- After encoding we get 64 dimensional feature which we feed to the back bone, as the encoding shape is different the back bone was modified to accommodate this change.
- Our Back bone has 3 layers same as the voxelnet but we used 2D backbone due the nature of the encoded feature.
- Once encoded, the features are scattered back to the original pillar locations to create a pseudo-image of size $(C,H,W)$ where $H$ and $W$ indicate the height and width of the canvas.
 

## Implementation

Detail of back bone

![](images/POWERPNT_yjVxjKIFZ4.png)

## Implementation

- Block 1
```python
    (0): Sequential(
      (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(64, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
      (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (4): BatchNorm2d(64, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
      (5): ReLU(inplace=True)
      (6): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (7): BatchNorm2d(64, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
      (8): ReLU(inplace=True)
      (9): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (10): BatchNorm2d(64, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
      (11): ReLU(inplace=True)
    )
```
- Block 2
```python
    (1): Sequential(
      (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(128, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
      (3): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (4): BatchNorm2d(128, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
      (5): ReLU(inplace=True)
      (6): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (7): BatchNorm2d(128, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
      (8): ReLU(inplace=True)
      (9): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (10): BatchNorm2d(128, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
      (11): ReLU(inplace=True)
      (12): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (13): BatchNorm2d(128, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
      (14): ReLU(inplace=True)
      (15): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (16): BatchNorm2d(128, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
      (17): ReLU(inplace=True)
    )
```
- Block 3
```python
    (2): Sequential(
      (0): Conv2d(128, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(256, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
      (3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (4): BatchNorm2d(256, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
      (5): ReLU(inplace=True)
      (6): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (7): BatchNorm2d(256, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
      (8): ReLU(inplace=True)
      (9): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (10): BatchNorm2d(256, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
      (11): ReLU(inplace=True)
      (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (13): BatchNorm2d(256, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
      (14): ReLU(inplace=True)
      (15): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (16): BatchNorm2d(256, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
      (17): ReLU(inplace=True)
    )
```

## Implementation

Detail of back bone

- For our specific implementation the image size at each block was as follows:
    - Block 1
      `in: [1, 64, 496, 432], out: [1, 64, 248, 216]`
    - Block 2
      `in: [1, 64, 248, 216], out: [1, 128, 124, 108]`
    - Block 3
      `in: [1, 128, 124, 108],out: [1, 256, 62, 54]`
- All these 3 block are fed to neck. The neck is very similar to Voxel net :


## Implementation 


- Neck concatenate all the 3 and produced a tensor with shape [1, 384, 248, 216] and this was fed to the loss calculation.
- we find focal loss, for class prediction, after class head ( class head similar to voxel net)
- we find bounding box loss after the class head.

## Implementation 

### Neck 

```python
    (0): Sequential(
      (0): ConvTranspose2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (1): BatchNorm2d(128, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
    )
    (1): Sequential(
      (0): ConvTranspose2d(128, 128, kernel_size=(2, 2), stride=(2, 2), bias=False)
      (1): BatchNorm2d(128, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
    )
    (2): Sequential(
      (0): ConvTranspose2d(256, 128, kernel_size=(4, 4), stride=(4, 4), bias=False)
      (1): BatchNorm2d(128, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
    )

```

## Experiments 

![](images/Code_mZ4BlvEcw3.png)

## Experiments 

![](images/Code_bzpSd5S747.png)

## Experiments 

![](images/Code_3jguzGHXJl.png)

## Experiments 

![](images/Code_VRuh7ylhGX.png)

## Experiments 

![](images/Code_yVGFxInDz1.png)

## Results : Quantitative  

![](images/EXCEL_0r5dQBNuZJ.png)

## Results : Qualitative

![](images/001733.png)
![](images/007065.png)
![](images/001666.png)
![](images/007402.png)
![](images/002079.png)
![](images/007067.png)
![](images/007403.png)
![](images/001817.png)
![](images/002058.png)
![](images/001931.png)
![](images/002424.png)
![](images/001774.png)
![](images/001979.png)
![](images/001740.png)
![](images/002035.png)
![](images/002153.png)
![](images/007232.png)
![](images/002232.png)
![](images/002348.png)
![](images/002362.png)
![](images/002418.png)
![](images/002612.png)
![](images/003677.png)



## Referenced papers

- [MVX-Net: Multimodal VoxelNet for 3D Object Detection](https://arxiv.org/pdf/1904.01649.pdf)^[MVX-Net: Multimodal VoxelNet for 3D Object Detection Vishwanath A. Sindagi, Yin Zhou, Oncel Tuzel 	arXiv:1904.01649 ]
- [ImVoxelNet: Image to Voxels Projection for Monocular and Multi-View General-Purpose 3D Object Detection](https://arxiv.org/pdf/2106.01178.pdf)^[ImVoxelNet: Image to Voxels Projection for Monocular and Multi-View General-Purpose 3D Object Detection Danila Rukhovich, Anna Vorontsova, Anton Konushin 	arXiv:2106.01178]
- [MVX-Net: Multimodal VoxelNet for 3D Object Detection](https://arxiv.org/pdf/1904.01649.pdf)^[MVX-Net: Multimodal VoxelNet for 3D Object Detection Vishwanath A. Sindagi, Yin Zhou, Oncel Tuzel 	arXiv:1904.01649 ]
- [Group-Free 3D Object Detection via Transformers](https://arxiv.org/pdf/2104.00678.pdf)^[Group-Free 3D Object Detection via Transformers  Ze Liu, Zheng Zhang, Yue Cao, Han Hu, Xin Tong 	arXiv:2104.00678]
- [End-to-End Multi-View Fusion for 3D Object Detection in LiDAR Point Clouds (Dynamic Voxelization)](https://arxiv.org/pdf/1910.06528.pdf)^[End-to-End Multi-View Fusion for 3D Object Detection in LiDAR Point Clouds Yin Zhou, Pei Sun, Yu Zhang, Dragomir Anguelov, Jiyang Gao, Tom Ouyang, James Guo, Jiquan Ngiam, Vijay Vasudevan 	arXiv:1910.06528]
- [PointNet: Deep Learning on Point Sets for 3D Classification and Segmentation](https://arxiv.org/abs/1612.00593)^[PointNet: Deep Learning on Point Sets for 3D Classification and Segmentation Charles R. Qi, Hao Su, Kaichun Mo, Leonidas J. Guibas 	arXiv:1612.00593]
- [PointPillars: Fast Encoders for Object Detection from Point Clouds](https://arxiv.org/abs/1812.05784)^[PointPillars: Fast Encoders for Object Detection from Point Clouds Alex H. Lang, Sourabh Vora, Holger Caesar, Lubing Zhou, Jiong Yang, Oscar Beijbom 	arXiv:1812.05784]

## Conclusion

- In this work we show that the performance of voxelnet can be improved by designing an improved version of voxel encoder. 
- The performance of the network highly depends on the input data encoding.
- we also show that the one classification head is sufficient, and only one network is sufficient. 

# $$\mathcal{THANKS!}$$